In [1]:
import re
import json
import string

import tweepy
import numpy as np
import pandas as pd
from time import sleep

import spacy
from textblob import TextBlob

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
# Consumer keys and access tokens, used for OAuth
consumer_key = 'ZE0w9dhnGHcWMKBqqvhQe2rYc'
consumer_secret = '96cnmIEGxDIsErs3HyyPXiayrPw9pedxNuOmwF9a3LMruCJaKl'
access_token = '1971207074-7jQ7P9e0dyDhVy0kMN3fva4PWGgtvI7c1xDXe2O'
access_token_secret = 'P3yKKm3okxM3ydYuAb57dD2uqcziZW7p9I3owndTFHA4G'

In [4]:
# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [5]:
# Creation of the actual interface, using authentication
api = tweepy.API(auth, wait_on_rate_limit = True)

In [6]:
account_dict = {
    'Donald Trump': 'realDonaldTrump',
    'Bernie Sanders': 'BernieSanders',
    'Barack Obama': 'BarackObama',
    'Joe Biden': 'JoeBiden',
    'Hillary Clinton': 'HillaryClinton',
    'Kamala Harris': 'KamalaHarris',
    'Rudy Giuliani': 'RudyGiuliani',
    'Ivanka Trump': 'IvankaTrump',
    'Bill Gates': 'BillGates',
    'Elon Musk': 'elonmusk'
}

In [7]:
def re_remove_url(x):
    return re.sub(r'(http|www.)\S+', '', x).replace('\n', '')

In [8]:
def extract_hashtags(x):
    try:
        hashtags = re.findall(r"#(\w+)", x) # Extract hashtags
        if not hashtags: return np.nan
        elif isinstance(hashtags, list): return ', '.join(hashtags) 
        else: return hashtags
    except:
        return np.nan

In [9]:
def extract_mentions(x):
    try:
        mentions = re.findall(r"@(\w+)", x) # Extract mentions
        if not mentions: return np.nan
        if isinstance(mentions, list): return ', '.join(mentions)
        else: return mentions
    except:
        return np.nan

In [10]:
def extract_text_details(x):
    length = len(x) # lenght if each tweet
    spaces = sum([1 for l in x if l.isspace()]) # how many spaces in each tweet
    uppers = sum([1 for l in x if l.isupper()]) # how many uppercases in each tweet
    punctuations  = sum([1 for l in x if l in string.punctuation]) # how many punctuations in each tweet
    questionsmark = x.count('?') # how many question marks in each tweet
    explainations = x.count('!') # how many explaination marks in each tweet
    return length, spaces, uppers, punctuations, questionsmark, explainations

In [11]:
def cleaning_tweets(x):
    try:
        x = str(x)
        # Spacy pipeline
        tweet = nlp(x)
        # Extract lemmatized words in lower case format if not digits, not punctuation, not stopword, and lenght not less than 2 
        tweet = ' '.join([toke8888/?token=e4605cac42ba1e4bcf36167299d35b578fa279a6be911786n.lemma_.lower() for token in tweet if not token.is_stop and not token.is_punct and not token.text.isdigit() and len(token.text) > 2])
        return tweet
    except:
        return np.nan

In [12]:
def polarity_status(x):
    if x == 0: return 'Neutral'
    elif x > 0.00: return 'Positive'
    elif x < 0.00: return 'Negative'
    else: return 'Unknown'

In [13]:
def subjectivity_status(x):
    if x <= 0.50: return 'Objective'
    elif x > 0.50: return 'Subjective'
    else: 'Unknown'

In [14]:
# Polarity and Subjectivity percentages and status
def polarity_subjectivity(x):
    try:
        x = str(x)
        analysis = TextBlob(x)
        polarity = round(analysis.polarity, 2)
        subjectivity   = round(analysis.subjectivity, 2)
        polarity_class = polarity_status(polarity)
        subjectivity_class = subjectivity_status(subjectivity)
        return polarity_class, subjectivity_class
    except:
        return np.nan, np.nan        

In [15]:
def part_of_the_day(x):
    try:
        if x in [5,6,7,8,9,10,11]: return 'Morning'
        elif x in [12,13,14,15,16]: return 'Afternoon' 
        elif x in [17,18,19]: return 'Evening'
        elif x in [20,21,22,23,24,1,2,3,4]: return 'Night'
        else: 'Unknown'
    except:
        return np.nan

In [16]:
def download_tweets(profile, name, num):
    
    tweets = api.user_timeline(screen_name = profile, count = num, include_rts = False, tweet_mode = 'extended')
    
    all_tweets = []
    all_tweets.extend(tweets)
    oldest_id = tweets[-1].id

    while True:
        tweets = api.user_timeline(screen_name = profile, count = num, include_rts = False, max_id = oldest_id - 1, tweet_mode = 'extended')
        if len(tweets) == 0:
            break
        oldest_id = tweets[-1].id
        all_tweets.extend(tweets)
    
    profile_name    = []
    created_date    = []
    created_hour    = []
    created_weekday = []
    favorite_count  = []
    retweet_count   = []
    source_device   = []
    tweet_text      = []
    clean_tweet     = []
    
    length_list     = []
    spaces_list     = []
    uppers_list     = []
    punctuations_list  = []
    questionsmark_list = []
    explainations_list = []
    
    day_part_list    = []
    hashtags_list    = []
    mentions_list    = []
    clean_tweet_list = []
    polarity_class_list = []
    subjectivity_class_list = []
        
    for tweet in all_tweets:
       
        try: profile_name.append(name) 
        except: profile_name.append(np.nan)
            
        try: created_date.append(tweet.created_at) 
        except: created_date.append(np.nan)
        
        try: created_hour.append(tweet.created_at.hour) 
        except: created_hour.append(np.nan)
        
        try: created_weekday.append(tweet.created_at.weekday()) 
        except: created_weekday.append(np.nan)
        
        try: favorite_count.append(tweet.favorite_count) 
        except: favorite_count.append(np.nan)
        
        try: retweet_count.append(tweet.retweet_count) 
        except: retweet_count.append(np.nan)
        
        try: source_device.append(tweet.source.split()[-1]) 
        except: source_device.append(np.nan)
        
        try: tweet_text.append(tweet.full_text) 
        except: tweet_text.append(np.nan)
        
        day_part    = part_of_the_day(tweet.created_at.hour)
        clean_tweet = re_remove_url(tweet.full_text)
        length, spaces, uppers, punctuations, questionsmark, explainations = extract_text_details(clean_tweet)
        hashtags    = extract_hashtags(clean_tweet)
        mentions    = extract_mentions(clean_tweet)
        clean_tweet = cleaning_tweets(clean_tweet)
        polarity_class, subjectivity_class = polarity_subjectivity(clean_tweet)
    
        length_list.append(length)
        spaces_list.append(spaces)
        uppers_list.append(uppers)
        punctuations_list.append(punctuations)
        questionsmark_list.append(questionsmark)
        explainations_list.append(explainations)
        
        day_part_list.append(day_part)
        hashtags_list.append(hashtags)
        mentions_list.append(mentions)
        clean_tweet_list.append(clean_tweet)
        polarity_class_list.append(polarity_class)
        subjectivity_class_list.append(subjectivity_class)
        
    dataframe_dict = {  
        'created_date': created_date,
        'profile_name': profile_name,
        'created_hour': created_hour, 
        'created_weekday': created_weekday,
        'favorite_count': favorite_count,
        'retweet_count': retweet_count,
        'source_device': source_device,
        'tweet_text': tweet_text,
        
        'length': length_list,
        'spaces': spaces_list,
        'uppers': uppers_list,
        'punctuations': punctuations_list,
        'questionsmark': questionsmark_list,
        'explainations': explainations_list,
        
        'day_part': day_part_list,
        'hashtags': hashtags_list,
        'mentions': mentions_list,
        'clean_tweet': clean_tweet_list,
        'polarity_class': polarity_class_list,
        'subjectivity_class': subjectivity_class_list
    }
    
    print(f'Done with {len(all_tweets)} for {name}')
    
    return pd.DataFrame(dataframe_dict)

In [17]:
dataframes_list = []

for name, profile in account_dict.items():
    dataframes_list.append(download_tweets(profile, name, 200))
    sleep(60)

Done with 1998 for Donald Trump
Done with 2596 for Bernie Sanders
Done with 2876 for Barack Obama
Done with 3008 for Joe Biden
Done with 2530 for Hillary Clinton
Done with 2682 for Kamala Harris
Done with 2346 for Rudy Giuliani
Done with 1808 for Ivanka Trump
Done with 3027 for Bill Gates
Done with 3003 for Elon Musk


In [45]:
df = pd.concat(dataframes_list).reset_index(drop = True)
df

,created_date,profile_name,created_hour,created_weekday,favorite_count,retweet_count,source_device,tweet_text,length,spaces,uppers,punctuations,questionsmark,explainations,day_part,hashtags,mentions,clean_tweet,polarity_class,subjectivity_class
0,2020-12-20 19:23:18,Donald Trump,19,6,68330,19058,iPhone,https://t.co/8S0ePCOCOG,0,0,0,0,0,0,Evening,NaN,NaN,NaN,Neutral,Objective
1,2020-12-20 17:51:12,Donald Trump,17,6,106968,32162,iPhone,Wow! https://t.co/gAttnzPa5d,5,1,1,1,0,1,Evening,NaN,NaN,wow,Positive,Subjective
2,2020-12-20 16:52:28,Donald Trump,16,6,220867,37540,iPhone,Cute! https://t.co/bMkuaxd7lL,6,1,1,1,0,1,Afternoon,NaN,NaN,cute,Positive,Subjective
3,2020-12-20 16:30:59,Donald Trump,16,6,104008,19277,iPhone,David is a great guy and patriot. Thank you La...,52,10,3,2,0,1,Afternoon,NaN,NaN,david great guy patriot thank lauren,Positive,Subjective
4,2020-12-20 05:26:49,Donald Trump,5,6,384045,80599,iPhone,GREATEST ELECTION FRAUD IN THE HISTORY OF OUR ...,56,8,45,3,0,3,Morning,NaN,NaN,greatest election fraud history country,Positive,Subjective
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25873,2020-02-09 06:45:25,Elon Musk,6,6,5032,167,iPhone,@Erdayastronaut First two domes in frame are f...,78,13,6,2,0,0,Morning,NaN,Erdayastronaut,@erdayastronaut dome frame sn2 sn1 thrust dome,Neutral,Objective
25874,2020-02-09 05:36:01,Elon Musk,5,6,70262,2392,iPhone,Back in Boca https://t.co/RjiWpW28PT,13,3,2,0,0,0,Morning,NaN,NaN,boca,Neutral,Objective
25875,2020-02-08 23:11:50,Elon Musk,23,5,25192,747,iPhone,@neiltyson Including hypothesizing that this s...,68,8,1,1,0,0,Night,NaN,neiltyson,@neiltyson include hypothesize statement wrong...,Negative,Subjective
25876,2020-02-08 23:10:37,Elon Musk,23,5,3064,68,iPhone,@flcnhvy True,13,1,1,1,0,0,Night,NaN,flcnhvy,@flcnhvy true,Positive,Subjective


In [19]:
df.to_csv('multiclass_text_classification.csv')